In [222]:
import pandas as pd
import numpy as np
import json
from bardapi import Bard
import os

from translate import Translator
from tqdm.auto import tqdm
import re
from difflib import SequenceMatcher

In [9]:
# import the json data
with open('Testdata.json') as f:
    data = json.load(f)
print(data)

[{'text': 'Je recherche une montre pour hommes de marque Tissot avec un prix inférieur à 1000DT.', 'entities': [{'entity': 'Category', 'value': 'montre'}, {'entity': 'Subcategory', 'value': 'montre pour hommes'}, {'entity': 'Marque', 'value': 'Tissot'}, {'entity': 'Prix', 'value': 'Inférieur à 1000DT'}]}, {'text': 'Je veux acheter des lunettes de soleil de marque Ray-Ban avec un prix entre 9 et 1420DT.', 'entities': [{'entity': 'Category', 'value': 'lunettes'}, {'entity': 'Subcategory', 'value': 'Lunettes de Soleil'}, {'entity': 'Marque', 'value': 'Ray-Ban'}, {'entity': 'Prix', 'value': 'Entre 9 et 1420DT'}]}, {'text': 'Je cherche un parfum pour femmes de la marque Chanel avec un volume de 50ml.', 'entities': [{'entity': 'Category', 'value': 'parfum'}, {'entity': 'Subcategory', 'value': 'parfum pour Femmes'}, {'entity': 'Marque', 'value': 'Chanel'}, {'entity': 'Volume', 'value': '50ml'}]}, {'text': 'Je souhaite trouver un sac de la marque JEEP avec un prix inférieur à 199DT.', 'entitie

## translating the data to english since bard doesn't support other languages

In [110]:
from translate import Translator

def translate_dataset(dataset):
    translator = Translator(from_lang="fr", to_lang="en")
    translated_dataset = []
    
    for item in tqdm(dataset):
        translated_item = {}
        
        # Translate the text value
        translated_item['text'] = translator.translate(item['text'])
        
        # Translate the key names and values in entities
        translated_item['entities'] = []
        for entity in item['entities']:
            translated_entity = {
                'entity': translator.translate(entity['entity']),
                'value': translator.translate(entity['value'])
            }
            translated_item['entities'].append(translated_entity)
        
        translated_dataset.append(translated_item)
    
    return translated_dataset

# Example usage
translated_dataset = translate_dataset(data)

100%|██████████| 16/16 [01:57<00:00,  7.36s/it]


In [143]:
def split_data(data):
    spl = int(len(data) * 0.2)
    print(len(data))
    prompt = data[:spl]
    eval = data[spl:]
    return prompt, eval
prompt, eval = split_data(translated_dataset)

16


In [ ]:
with open('WebsiteCategoriesDataNoLink.json') as f:
   WebCategs = json.load(f)


## extracting the categories from the data

In [142]:
#extract website keys from the json data
with open('WebsiteCategoriesDataNoLink.json') as f:
   WebCategs = json.load(f)

def get_unique_keys(dictionary):
    unique_keys = set()
    def extract_keys(obj):
        if isinstance(obj, dict):
            for key in obj:
                unique_keys.add(key)
                extract_keys(obj[key])
        elif isinstance(obj, list):
            for item in obj:
                extract_keys(item)
    extract_keys(dictionary)
    return list(unique_keys)

keys = get_unique_keys(WebCategs)
keys

['name',
 'subcategories',
 'categories',
 'Catégories',
 'Prix',
 'Pointure',
 'Marque',
 'Volume']

In [149]:
translatedKeys = []
translator = Translator(from_lang="fr", to_lang="en")
for key in keys:
    translatedKeys.append(translator.translate(key))

translatedKeys = translatedKeys[1:]
translatedKeys.pop(1)


['subcategories', 'categories', 'Categories', 'Price', 'Shoe size', 'Brand ', 'Volume']


'categories'

In [152]:
translatedKeys = ["Category", "subcategory", "Brand", "Price", "volume","shoe size"]

['Category', 'subcategory', 'Brand', 'price', 'volume', 'shoe size']


## preparing the prompt

In [244]:
def prepare_prompt(prompt, list_of_keys):
    prompt = prompt.copy()
    print(prompt)
    prp = f""""Extract the relevant information from the following user query, these are the possible categories: \n {list_of_keys}\n please just give me the answer for the last query only. give me the answer directly by following the pattern bellow, and make sure to follow the same writing pattern \nhere are some examples of the query and the relevant information to extract: \n"""
    for i in prompt:
        prp += f"""\nQuery: {i['text'].strip()}\n"""
        for key in i['entities']:
            prp += f"""**{key['entity'].strip()}** {key['value'].strip()}\n"""
    return prp
def prep_fewshot(prompt, ele):
    prp = prompt
    prp += f"""\nQuery: {ele['text'].strip()}\n **Category** """
    return prp

val = prepare_prompt(prompt, translatedKeys)

[{'text': "I am looking for a men's watch from Tissot with a price below 1000DT.", 'entities': [{'entity': 'Category', 'value': 'shows'}, {'entity': 'SUBCATEGORY', 'value': "men's watch"}, {'entity': 'Brand ', 'value': 'Tissot'}, {'entity': 'Price', 'value': 'Less than 1000DT'}]}, {'text': 'I want to buy Ray-Ban brand sunglasses with a price between 9 and 1420DT.', 'entities': [{'entity': 'Category', 'value': 'eyewear'}, {'entity': 'SUBCATEGORY', 'value': 'Sunglasses'}, {'entity': 'Brand ', 'value': 'Rayban'}, {'entity': 'Price', 'value': 'Between 9 and 1420DT'}]}, {'text': 'I am looking for a perfume for women from Chanel with a volume of 50ml.', 'entities': [{'entity': 'Category', 'value': 'perfume'}, {'entity': 'SUBCATEGORY', 'value': 'perfume for women'}, {'entity': 'Brand ', 'value': 'Chanel'}, {'entity': 'Volume', 'value': '50 ml'}]}]


In [154]:
print(val)

"Extract the relevant information from the following user query, these are the possible categories: 
 ['Category', 'subcategory', 'Brand', 'price', 'volume', 'shoe size']
 please just give me the answer for the last query only. give me the answer directly by following the pattern bellow, and make sure to follow the same writing pattern 
here are some examples of the query and the relevant information to extract: 

Query: I am looking for a men's watch from Tissot with a price below 1000DT.
**Category** shows
**SUBCATEGORY** men's watch
**Brand** Tissot
**Price** Less than 1000DT

Query: I want to buy Ray-Ban brand sunglasses with a price between 9 and 1420DT.
**Category** eyewear
**SUBCATEGORY** Sunglasses
**Brand** Rayban
**Price** Between 9 and 1420DT

Query: I am looking for a perfume for women from Chanel with a volume of 50ml.
**Category** perfume
**SUBCATEGORY** perfume for women
**Brand** Chanel
**Volume** 50 ml

Query: I want to find a JEEP brand bag with a price below 199DT.
 

# testing bard with the prompt

In [251]:
os.environ['_BARD_API_KEY'] = "API KEY"
response = Bard().get_answer(val)
print(response['content'])

Sure, here is the relevant information extracted from the query "I am looking for a perfume for women from Chanel with a volume of 50ml":

**Category:** Perfume
**Subcategory:** Perfume for women
**Brand:** Chanel
**Volume:** 50 ml


In [232]:
SequenceMatcher(None, "bags", "bag").ratio()

0.8571428571428571

## output evaluation 

In [230]:

import Levenshtein as lev

def evaluation(response, ele):
    correctElements = 0
    totalElements = 0
    response = response.splitlines()
    response = response[1:-1]
    response = "".join(response)
    response_tokens = response.replace('**',':').replace('\n',':').split(':')
    for entity in ele['entities']:
        if entity['value'].lower() in response.lower():
            correctElements += 1
        else:
             for token in response_tokens:
                if SequenceMatcher(None, entity['value'].lower(), token.lower()).ratio() > 0.5:
                    correctElements += 1
                    break
        totalElements += 1
    return correctElements, totalElements

In [245]:
print(val)

"Extract the relevant information from the following user query, these are the possible categories: 
 ['Category', 'subcategory', 'Brand', 'price', 'volume', 'shoe size']
 please just give me the answer for the last query only. give me the answer directly by following the pattern bellow, and make sure to follow the same writing pattern 
here are some examples of the query and the relevant information to extract: 

Query: I am looking for a men's watch from Tissot with a price below 1000DT.
**Category** shows
**SUBCATEGORY** men's watch
**Brand** Tissot
**Price** Less than 1000DT

Query: I want to buy Ray-Ban brand sunglasses with a price between 9 and 1420DT.
**Category** eyewear
**SUBCATEGORY** Sunglasses
**Brand** Rayban
**Price** Between 9 and 1420DT

Query: I am looking for a perfume for women from Chanel with a volume of 50ml.
**Category** perfume
**SUBCATEGORY** perfume for women
**Brand** Chanel
**Volume** 50 ml



## evaluating the output with accuracy

In [ ]:
correctElements = 0
totalElements = 0
for ele in tqdm(eval):
    response = Bard().get_answer(prep_fewshot(val, ele))
    correct, total = evaluation(response['content'], ele)
    correctElements += correct
    totalElements += total

In [253]:
print(f"Accuracy: {correctElements/totalElements}")

Accuracy: 0.82
